In [1]:
%load_ext autoreload
%autoreload 2
import pickle
import open3d as o3d
import numpy as np
test = None
with open("src/no_ground_all.pkl", "rb") as f:
    test = pickle.load(f)
pkl_downpcd = o3d.geometry.PointCloud()
pkl_downpcd.points = o3d.utility.Vector3dVector(test)

# eps= 3.0
# min_points = 3
eps= 2.5
min_points = 10
print(eps, min_points)

with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        pkl_downpcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))

max_label = labels.max()
test.shape

2.5 10
[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 2848


(1037625, 3)

In [29]:
def display_2d(data):
    import plotly.graph_objects as go
    from PIL import Image
    small_img = Image.open("../lidar/data/481E_5456N_tiny.png")
    size = 1000
    fig = go.Figure(data=[go.Scattergl(
    x = data[:, 0],
    y = data[:, 1],
    mode='markers',
    marker=dict(
        size=2,
        color=data[:, 2],                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=1.0
    ))])
    
    offset = 1000
    fig.add_layout_image(
            dict(
                source=small_img,
                xref="x",
                yref="y",
                x=0,
                y=offset,
                sizex=size,
                sizey=size,
                sizing="stretch",
                opacity=0.6,
                layer="below")
    )

    # Set templates
    fig.update_layout(template="plotly_white")
    fig.update_yaxes(
        scaleanchor = "x",
        scaleratio = 1,
      )
    fig.show()
    
def display_labelled_data_2d(data, labels):
    import plotly.graph_objects as go
    from PIL import Image
    small_img = Image.open("../lidar/data/481E_5456N_tiny.png")

    size = 1000
    fig = go.Figure()
    for i in np.arange(labels.max()):
        x_cluster = data[:, 0][np.where(labels == i)]
        y_cluster = data[:, 1][np.where(labels == i)]
        fig.add_trace(go.Scattergl(x=x_cluster, y=y_cluster,
                                    mode='markers',
                                    marker=dict(
                                        size=2,
                                        color=i,                # set color to an array/list of desired values
                                        colorscale='Viridis',   # choose a colorscale
                                        opacity=0.8
    )))
    
    offset = 1000
    fig.add_layout_image(
            dict(
                source=small_img,
                xref="x",
                yref="y",
                x=0,
                y=offset,
                sizex=size,
                sizey=size,
                sizing="stretch",
                opacity=0.8,
                layer="below")
    )

    # Set templates
    fig.update_layout(template="plotly_white")
    fig.update_yaxes(
        scaleanchor = "x",
        scaleratio = 1,
      )
    fig.show()
import plotly.io as pio
# too many points, cannot render embedded
pio.renderers.default = "browser"

# display_labelled_data_2d(np.asarray(pkl_downpcd.points), labels)
display_2d(np.asarray(pkl_downpcd.points))

In [8]:
from PIL import Image
import math
import plotly.graph_objects as go
small_img = Image.open("../lidar/data/481E_5456N_tiny.png")

data = np.asarray(pkl_downpcd.points)
img = np.array(small_img)
print(small_img.mode)
size = 1000
fig = go.Figure()

high_vegetation = None

for i in np.arange(labels.max()):
#     i = labels.max() - i
    x_cluster = data[:, 0][np.where(labels == i)]
    y_cluster = data[:, 1][np.where(labels == i)]
    z_cluster = data[:, 2][np.where(labels == i)]
    rgbvi_sum = 0
    gli_sum = 0
    for i in np.arange(x_cluster.shape[0]):
        rgb = img[math.floor(999 - y_cluster[i]), math.ceil(x_cluster[i]) - 1,  :]
        # the default type is uint
        rgb = rgb.astype('float64') 
        gli_sum += (2 * rgb[1] - rgb[0] - rgb[2]) / (2 * rgb[1] + rgb[0] + rgb[2])
        rgbvi_sum +=((rgb[1] * rgb[1] - rgb[0]*rgb[2]) * 1.0) /(rgb[1] * rgb[1]  + rgb[0]*rgb[2]) 
    rgbvi_avg = rgbvi_sum /  x_cluster.shape[0]
    gli_avg = gli_sum /  x_cluster.shape[0]
    # filter
    if rgbvi_avg > 0.03:
#     fig.add_annotation(x=x_cluster[0], y=y_cluster[0],
#         text="rgbvi:%.3f" % rgbvi_avg,
#         showarrow=True,
#         arrowhead=1)
#     fig.add_annotation(x=x_cluster[-1], y=y_cluster[-1],
#         text="gli:%.3f" % gli_avg,
#         showarrow=True,
#         arrowhead=1)
        if high_vegetation is None:
            high_vegetation = np.vstack((x_cluster, y_cluster, z_cluster)).T
        else:
            high_vegetation = np.vstack((high_vegetation, np.vstack((x_cluster, y_cluster, z_cluster)).T))
        fig.add_trace(go.Scattergl(x=x_cluster, y=y_cluster, 
           mode = "markers",
            marker=dict(
                size=3,
                color=i,                # set color to an array/list of desired values
                colorscale='Viridis',   # choose a colorscale
                opacity=0.8
        )))

offset = 1000
fig.add_layout_image(
        dict(
            source=small_img,
            xref="x",
            yref="y",
            x=0,
            y=offset,
            sizex=size,
            sizey=size,
            sizing="stretch",
            opacity=0.85,
            layer="below")
)

# Set templates
fig.update_layout(template="plotly_white")
fig.update_traces(textposition='top center')
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
# pio.renderers.default = "jupyterlab"
fig.show()

RGB


<ipython-input-8-fd15d7fc2861>:26: RuntimeWarning:

invalid value encountered in double_scalars



In [9]:
display_2d(high_vegetation)

In [27]:
import laspy
header = laspy.header.Header()
outFile1 = laspy.file.File("../lidar/data/2015/4810E_54560N.las", mode = "w",header = header)
outFile1.X = high_vegetation[:, 0]  + (4810 - 4000) * 10000
outFile1.Y = high_vegetation[:, 1]  + (54560 - 50000) * 10000
outFile1.Z = high_vegetation[:, 2] * 100
outFile1.Classification = np.ones(high_vegetation.shape[0]).astype(np.uint8) * 5

outFile1.close()

In [28]:
inFile = laspy.file.File("../lidar/data/2015/4810E_54560N.las", mode='r')
I = inFile.Classification == 5
test = inFile.points[I]
test

array([((8100118, 45600500,  240, 0, 0, 5, 0, 0, 0),),
       ((8100124, 45600500,  245, 0, 0, 5, 0, 0, 0),),
       ((8100123, 45600499,  488, 0, 0, 5, 0, 0, 0),), ...,
       ((8100741, 45600516,  961, 0, 0, 5, 0, 0, 0),),
       ((8100739, 45600514, 1160, 0, 0, 5, 0, 0, 0),),
       ((8100741, 45600515, 1014, 0, 0, 5, 0, 0, 0),)],
      dtype=[('point', [('X', '<i4'), ('Y', '<i4'), ('Z', '<i4'), ('intensity', '<u2'), ('flag_byte', 'u1'), ('raw_classification', 'u1'), ('scan_angle_rank', 'i1'), ('user_data', 'u1'), ('pt_src_id', '<u2')])])

In [13]:
inFile = laspy.file.File("../lidar/data/2018/4810E_54560N.las", mode='r')
I = inFile.Classification == 5
test = inFile.points[I]
test

array([((8107526, 45683767, 8986, 27655, 25, 5,  29, 1, 1015, 2.19477351e+08),),
       ((8107488, 45683637, 8751, 16471, 26, 5,  29, 1, 1015, 2.19477351e+08),),
       ((8107604, 45684041, 9349, 35432,  9, 5,  29, 1, 1015, 2.19477351e+08),),
       ...,
       ((8170733, 45607069, 8501,  1660, 26, 5, -17, 2, 2043, 2.19561417e+08),),
       ((8170807, 45607087, 8547, 16427, 25, 5, -17, 2, 2043, 2.19561417e+08),),
       ((8170783, 45607081, 8468, 23068, 26, 5, -17, 2, 2043, 2.19561417e+08),)],
      dtype=[('point', [('X', '<i4'), ('Y', '<i4'), ('Z', '<i4'), ('intensity', '<u2'), ('flag_byte', 'u1'), ('raw_classification', 'u1'), ('scan_angle_rank', 'i1'), ('user_data', 'u1'), ('pt_src_id', '<u2'), ('gps_time', '<f8')])])